QR code generation with Stable Diffusion

In [ ]:
!pip install qrcode==7.4.2

In [ ]:
import torch
import qrcode
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image
import os
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Create the directory if it doesn't exist
folder_path = "/content/drive/MyDrive/qr_codes"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
qr_link="https://github.com/nsourlos/coding_session_AI_tools"

prompt="colourful flower shop" #"Imax hypnosis graffiti graffiti sampler abstracts scanned acry, high quality"
# (one male engineer), medium curly hair, from side, (mechanics), circuit board, steampunk, machine, studio, table, science fiction, high contrast, high key, cinematic light,
# (masterpiece, top quality, best quality, official art, beautiful and aesthetic:1.3), extreme detailed, highest detailed, (ultra-detailed)

negative_prompt="""ugly, disfigured, blurry, nsfw, (worst quality, low quality:2), overexposure, watermark, text, easynegative, ugly, (blurry:2), bad_prompt,bad-artist, \
                    bad hand, ng_deepnegative_v1_75t"""
# (worst quality, low quality:2), overexposure, watermark, text, easynegative, ugly, (blurry:2), bad_prompt,bad-artist, bad hand, ng_deepnegative_v1_75t

In [ ]:
# "monster-labs/control_v1p_sd15_qrcode_monster" or "DionTimmer/controlnet_qrcode-control_v1p_sd15"
controlnet = ControlNetModel.from_pretrained("monster-labs/control_v1p_sd15_qrcode_monster", torch_dtype=torch.float16)

In [ ]:
pipe=StableDiffusionControlNetPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None).to("cuda")

In [ ]:
def generate_qr_code(url): #920*920 recommended - Inspired from: https://www.youtube.com/watch?v=y-rCMjxVrGA&ab_channel=AbhishekThakur
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H, #30% of errors can be corrected
        box_size=20,
        border=2)

    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert('RGB')

    #If we want to have it as 920x920
    # # Force final size to be exactly 920x920
    # w = h = 920
    # bg = Image.new("RGB", (w, h), "white")

    # # Center the QR code
    # coords = ((w - img.size[0]) // 2 // 16 * 16, (h - img.size[1]) // 2 // 16 * 16)
    # bg.paste(img, coords)

    #Alternative method that also worked - Only tested with num_steps, prompt, and img as parameters below
    # def resize_for_condition_image(input_image: Image, resolution: int):
    # input_image = input_image.convert("RGB")
    # W, H = input_image.size
    # k = float(resolution) / min(H, W)
    # H *= k
    # W *= k
    # H = int(round(H / 64.0)) * 64
    # W = int(round(W / 64.0)) * 64
    # img = input_image.resize((W, H), resample=Image.LANCZOS)
    # return img

    # qr_image = resize_for_condition_image(qr_image, 512)

    offset_min = 2*16
    w, h = img.size
    print("Original size:", w, h)
    w= (w+255+offset_min)//256*256
    h= (h+255+offset_min)//256*256
    print("New size:", w, h)
    if w>1024:
        raise ValueError("QR code too large")
    bg = Image.new("RGB", (w, h), "white")

    coords=((w-img.size[0])//2//16*16, (h-img.size[1])//2//16*16)
    bg.paste(img, coords)

    return bg

In [ ]:
qr_code_image=generate_qr_code(qr_link)

- Strength parameter defines how much noise is added to your QR code and the noisy QR code is then guided towards both your prompt and the QR code image via Controlnet. Use a high strength value between 0.8 and 0.95.
- Choose a conditioning scale between 0.6 and 2.0. This mode arguably achieves the asthetically most appealing QR code images, but also requires more tuning of the controlnet conditioning scale and the strength value.
- If the generated image looks way to much like the original QR code, make sure to gently increase the strength value and reduce the conditioning scale

In [ ]:
generator=torch.Generator()

In [ ]:
controlnet_conditioning_scales=[1.5, 1.8, 2.2]

guidance_scales = [5, 7, 12, 15, 17] 

for guidance_scale in guidance_scales:
    for controlnet_conditioning_scale in controlnet_conditioning_scales:
      # for strength_scale in strength_scales:
        for eta in [0.4, 0.7, 1]: #Doesn't seem to have a big impact
          for inf_steps in [20, 40]:

            print("gs_"+str(guidance_scale)+"_ccs_"+str(controlnet_conditioning_scale)+"_eta_"+str(eta)+"_steps_"+str(inf_steps))

            # Generate image - https://huggingface.co/DionTimmer/controlnet_qrcode-control_v1p_sd15
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=qr_code_image,
                width=qr_code_image.width,
                height=qr_code_image.height,
                guidance_scale=float(guidance_scale),
                controlnet_conditioning_scale=float(controlnet_conditioning_scale),
                generator=generator,
                seed=42, #For reproducibility
                eta=float(eta),
                num_inference_steps=inf_steps
                # strength=float(strength_scale), #default 0.9
            ).images[0]

            # Display and save the generated image
            image.show()
            image.save(os.path.expanduser(folder_path+"/generated_qr_gs_"+str(guidance_scale)+"_ccs_"+str(controlnet_conditioning_scale)+ \
                                          "_eta_"+str(eta)+"_steps_"+str(inf_steps)+".png"))

In [ ]:
#Zip and download folder from google drive
!zip -r /content/qr_codes.zip /content/drive/MyDrive/qr_codes
#Download zipped folder
from google.colab import files
files.download("/content/qr_codes.zip")